In [1]:
!pip install -q pymupdf pdfplumber sentence-transformers faiss-cpu tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 73.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 98.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.9 MB/s eta 0:00:00:00:01


In [2]:
# ================================
# Gen-AI PDF Summarizer (NotebookLM-inspired)
# Environment Setup Cell
# ================================

# Core
import os
import re
import json
import math
from typing import List, Dict

# Data handling
import numpy as np
import pandas as pd

# PDF processing
import fitz  # PyMuPDF
import pdfplumber

# NLP & Embeddings
from sentence_transformers import SentenceTransformer

# Vector Search
import faiss

# Token counting (important for chunking)
import tiktoken

# LLM API (choose ONE later)
# import openai
# from transformers import pipeline

# Visualization / Debug
from tqdm import tqdm
from pprint import pprint

print("✅ Environment setup complete")


2026-01-29 15:05:40.497320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769699140.684576      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769699140.739495      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769699141.196588      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769699141.196623      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769699141.196625      55 computation_placer.cc:177] computation placer alr

✅ Environment setup complete


In [3]:
# ================================
# Improved Embeddings + Re-indexing
# (BGE model + context prefix)
# ================================

# Stronger embedding model (much better for retrieval)
embedding_model = SentenceTransformer(
    "BAAI/bge-base-en-v1.5"
)

print("✅ BGE embedding model loaded")

# BGE requires query & passage prefixes
PASSAGE_PREFIX = "Represent this passage for retrieval: "
QUERY_PREFIX = "Represent this question for retrieving relevant passages: "

# Prepare texts WITH context prefix
texts = []
for chunk in chunks:
    contextual_text = (
        f"Page {chunk['page']}. "
        + chunk["text"].strip()
    )
    texts.append(PASSAGE_PREFIX + contextual_text)

# Generate embeddings
embeddings = embedding_model.encode(
    texts,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print(f"✅ Generated embeddings with shape: {embeddings.shape}")

# Rebuild FAISS index
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatIP(embedding_dim)
index.add(embeddings)

print(f"✅ FAISS index rebuilt with {index.ntotal} vectors")

# Keep metadata aligned
chunk_metadata = chunks

# Sanity check
print("\n🔍 Sample embedded passage:\n")
print(texts[0][:300], "...")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ BGE embedding model loaded


NameError: name 'chunks' is not defined

In [ ]:
# ================================
# PDF Loading & Basic Inspection
# ================================

# Update this to your dataset folder name if needed
DATA_DIR = "/kaggle/input"

# Find the first PDF in the input directory
pdf_path = None
for root, dirs, files in os.walk(DATA_DIR):
    for file in files:
        if file.lower().endswith(".pdf"):
            pdf_path = os.path.join(root, file)
            break
    if pdf_path:
        break

if pdf_path is None:
    raise FileNotFoundError("❌ No PDF file found in /kaggle/input")

print(f"📄 PDF found at: {pdf_path}")

# Open PDF
doc = fitz.open(pdf_path)

print(f"✅ PDF loaded successfully")
print(f"📑 Total pages: {doc.page_count}")

# Preview text from first page
first_page = doc.load_page(0)
preview_text = first_page.get_text("text")[:1000]

print("\n🔍 Preview of Page 1:\n")
print(preview_text if preview_text.strip() else "[No extractable text found]")


In [ ]:
# ================================
# Page-wise Text Extraction
# ================================

def clean_text(text: str) -> str:
    """
    Light text cleaning:
    - Remove excessive newlines
    - Remove repeated spaces
    """
    text = re.sub(r'\n{2,}', '\n', text)
    text = re.sub(r'[ \t]{2,}', ' ', text)
    return text.strip()


pages_data = []

for page_num in tqdm(range(doc.page_count), desc="Extracting pages"):
    page = doc.load_page(page_num)
    text = page.get_text("text")
    
    if text and text.strip():
        cleaned_text = clean_text(text)
    else:
        cleaned_text = ""

    pages_data.append({
        "page": page_num + 1,   # 1-based indexing (human friendly)
        "text": cleaned_text
    })

print(f"✅ Extracted text from {len(pages_data)} pages")

# Show sample output
print("\n🔍 Sample extracted page data:\n")
pprint(pages_data[0])


In [ ]:
# ================================
# Token-aware Text Chunking
# ================================

# Initialize tokenizer (OpenAI-style tokenization works well even for open-source models)
tokenizer = tiktoken.get_encoding("cl100k_base")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))


def chunk_text(
    text: str,
    max_tokens: int = 700,
    overlap: int = 100
) -> List[str]:
    """
    Splits text into overlapping token-based chunks.
    """
    tokens = tokenizer.encode(text)
    chunks = []

    start = 0
    while start < len(tokens):
        end = start + max_tokens
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
        start = end - overlap

    return chunks


# Create chunks with page metadata
chunks = []

for page_data in tqdm(pages_data, desc="Chunking pages"):
    page_text = page_data["text"]
    
    if not page_text:
        continue

    page_chunks = chunk_text(page_text)

    for idx, chunk in enumerate(page_chunks):
        chunks.append({
            "page": page_data["page"],
            "chunk_id": f"page_{page_data['page']}_chunk_{idx}",
            "text": chunk,
            "tokens": count_tokens(chunk)
        })

print(f"✅ Total chunks created: {len(chunks)}")

# Show sample chunk
print("\n🔍 Sample chunk:\n")
pprint(chunks[0])


In [ ]:
# ================================
# Embeddings + FAISS Vector Index
# ================================

# Load embedding model (GPU will be used automatically if available)
embedding_model = SentenceTransformer(
    "sentence-transformers/all-MiniLM-L6-v2"
)

print("✅ Embedding model loaded")

# Prepare texts for embedding
texts = [chunk["text"] for chunk in chunks]

# Generate embeddings
embeddings = embedding_model.encode(
    texts,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print(f"✅ Generated embeddings with shape: {embeddings.shape}")

# Create FAISS index
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatIP(embedding_dim)  # Inner Product (cosine similarity with normalized vectors)

index.add(embeddings)

print(f"✅ FAISS index created with {index.ntotal} vectors")

# Keep metadata aligned with index positions
chunk_metadata = chunks  # same order as embeddings

# Sanity check
print("\n🔍 Sample metadata entry:\n")
pprint(chunk_metadata[0])


In [ ]:
# ================================
# Semantic Retrieval (RAG Step 1)
# ================================

def retrieve_relevant_chunks(
    question: str,
    top_k: int = 5
) -> List[Dict]:
    """
    Retrieves top-k most relevant chunks for a given question.
    """
    # Embed the question
    question_embedding = embedding_model.encode(
        [question],
        convert_to_numpy=True,
        normalize_embeddings=True
    )

    # Search FAISS index
    scores, indices = index.search(question_embedding, top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        chunk = chunk_metadata[idx]
        results.append({
            "score": float(score),
            "page": chunk["page"],
            "chunk_id": chunk["chunk_id"],
            "text": chunk["text"][:500] + "..."  # preview only
        })

    return results


# -------------------------------
# Test retrieval
# -------------------------------
sample_question = "What methodology does the paper propose and how does it work?"


retrieved_chunks = retrieve_relevant_chunks(sample_question, top_k=5)

print(f"\n🔎 Question: {sample_question}\n")
for i, res in enumerate(retrieved_chunks, 1):
    print(f"Result {i}")
    print(f"  Score : {res['score']:.4f}")
    print(f"  Page  : {res['page']}")
    print(f"  Chunk : {res['chunk_id']}")
    print(f"  Text  : {res['text']}\n")


In [4]:
# ================================
# Two-Stage Retrieval + Re-ranking
# ================================

def is_reference_chunk(text: str) -> bool:
    text_lower = text.lower()

    reference_signals = [
        "references",
        "bibliography",
        "proceedings of",
        "doi:",
        "pp.",
        "et al.",
        "association for computational linguistics",
        "conference",
        "journal"
    ]

    # If many reference signals appear, it's likely a citation block
    hits = sum(1 for sig in reference_signals if sig in text_lower)
    return hits >= 2


INTENT_KEYWORDS = [
    "method", "methodology", "approach",
    "model", "architecture", "framework",
    "proposed", "system"
]

def keyword_score(text: str) -> int:
    text_lower = text.lower()
    return sum(1 for kw in INTENT_KEYWORDS if kw in text_lower)


def retrieve_relevant_chunks_v2(
    question: str,
    stage1_k: int = 15,
    final_k: int = 5,
    min_score: float = 0.3
):
    """
    Two-stage retrieval:
    1) Broad FAISS search
    2) Re-rank using similarity + keyword + length heuristics
    """

    # Embed question (BGE requires prefix)
    question_embedding = embedding_model.encode(
        [QUERY_PREFIX + question],
        convert_to_numpy=True,
        normalize_embeddings=True
    )

    # Stage 1: broad retrieval
    scores, indices = index.search(question_embedding, stage1_k)

    candidates = []
    for score, idx in zip(scores[0], indices[0]):
        if score < min_score:
            continue

        chunk = chunk_metadata[idx]
        text = chunk["text"]

        if is_reference_chunk(text):
            continue


        candidates.append({
            "score": float(score),
            "page": chunk["page"],
            "chunk_id": chunk["chunk_id"],
            "text": text,
            "keyword_score": keyword_score(text),
            "length": len(text)
        })

    if not candidates:
        return []

    # Stage 2: re-ranking
    candidates.sort(
        key=lambda x: (
            x["score"],            # semantic similarity
            x["keyword_score"],    # intent alignment
            x["length"]            # prefer richer chunks
        ),
        reverse=True
    )

    return candidates[:final_k]


# -------------------------------
# Test improved retrieval
# -------------------------------
test_question = "What methodology does the paper propose and how does it work?"

results = retrieve_relevant_chunks_v2(test_question)

print(f"\n🔎 Question: {test_question}\n")

if not results:
    print("❌ No strong matches found.")
else:
    for i, res in enumerate(results, 1):
        print(f"Result {i}")
        print(f"  Score         : {res['score']:.4f}")
        print(f"  Keyword score : {res['keyword_score']}")
        print(f"  Page          : {res['page']}")
        print(f"  Chunk         : {res['chunk_id']}")
        print(f"  Text preview  : {res['text'][:500]}...\n")


NameError: name 'index' is not defined